In [9]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "syntheticMRI3D"
FINAL_DATA_NAME = 'syntheticMRI3D-full-wavelet' # + channel
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "syntheticMRI3D-cleaned-jitter"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','syntheticMRI3D')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir

'/Users/brandonmarks/Desktop/hierarchical-bayesian-model-validation/raw-data/syntheticMRI3D'

In [10]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [11]:
file_list = [os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}", filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(os.path.join(data_dir, f"full-{RAW_DATA_SUFFIX}"))
file_names[:5]

['sub-018747_sub-018747_T1w_cleaned.npz',
 'sub-090003_sub-090003_T1w_cleaned.npz',
 'sub-013808_sub-013808_T1w_cleaned.npz',
 'sub-074180_sub-074180_T1w_cleaned.npz',
 'sub-017913_sub-017913_T1w_cleaned.npz']

In [12]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


full-syntheticMRI3D-cleaned-jitter



# Wavelet

In [13]:

batch_dir = os.path.join(ROOT_DIR, "raw-data", 'syntheticMRI3D', f"full-{RAW_DATA_SUFFIX}")


In [14]:
TRANSFORM = "wavelet"


channel_wv = convert_to_wavelet_basis_3d(batch_dir, debug = True)
channel_wv['data'] = channel_wv['data'].apply(lambda x: x[~np.isnan(x)])
channel_wv['data'] = channel_wv['data'].apply(lambda x : x.astype(np.float32))
orientations = np.unique(channel_wv['orientation'].to_numpy()[1:])
print(orientations)
if SAVE_DF:
   
    pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME}-{channel}-df.pickle"))
    
min_group, max_group = 2, max(channel_wv['layer'])
for orientation in orientations:
    group_data_map = dict()
    group_data_map_size = dict()
    for group in np.arange(min_group, max_group + 1):
        data = np.array(channel_wv[(channel_wv['orientation'] == orientation) & (channel_wv['layer'] == group)]['data'].iloc[0])
        data = np.append(data, -data)
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
        group_data_map_size[group] = data.size
        freq_df.loc[DATASET, TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group)]['frequency'].iloc[0]
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"))
                                
    
    pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{orientation}.pickle"))
    pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{orientation}-size.pickle"))

channel_wv.head()

8 layers being used


  0%|          | 0/299 [00:00<?, ?it/s]

['aad' 'ada' 'add' 'daa' 'dad' 'dda' 'ddd']


,layer,frequency,orientation,data
0,1,0.996109,L1,[]
1,2,0.498054,aad,[]
2,2,0.498054,ada,[]
3,2,0.498054,add,[]
4,2,0.498054,daa,[]


In [15]:
del channel_wv, group_data_map, group_data_map_size

In [16]:
FINAL_DATA_NAME


'syntheticMRI3D-full-wavelet'